In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/tfp_anomaly_detection.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/tfp_anomaly_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Anomaly Detection with TensorFlow Probability STS on Kubeflow Pipelines

## Overview

This notebook demonstrates how to use [TensorFlow Probability](https://www.tensorflow.org/probability) and [Kubeflow Pipelines](https://www.kubeflow.org/docs/components/pipelines/) for anomaly detection in time series data. It uses structural time series (STS), a class of Bayesian statistical models, to decompose a time series into interpretable seasonal and trend components. This algorithm fits an STS model to the time series, generates a forecast of acceptable values for each timestep, and flags any points outside of the forecast as an anomaly. To learn more about STS models, check out this demo on [Structural Time Series Modeling Case Studies](https://www.tensorflow.org/probability/examples/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand).

This demo is most relevant for those who would like to automatically flag anomalies in time series data and can be used for applications like network monitoring, infrastructure maintenance, and sales tracking.

### Dataset

This demo uses the [Numenta Anomaly Benchmark](https://github.com/numenta/NAB), a popular benchmark of time series data with labeled anomalies. More specifically, our demo uses [nyc_taxi.csv](https://github.com/numenta/NAB/blob/d2854d17a3feb9e143b1e9a715c5af67da2c1888/data/realKnownCause/nyc_taxi.csv) which reports the total number of passengers in NYC taxis from July 2014 to January 2015 in 30-minute increments.

### Objective

You will go through the following steps: 
* Define and launch an anomaly detection algorithm on Kubeflow Pipelines.
* Retrieve and visualize results.
* Benchmark predictions using the Numenta Anomaly Benchmark scoring method.

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install {USER_FLAG} --upgrade kfp
! pip3 install {USER_FLAG} --upgrade google-cloud-pipeline-components
! pip3 install {USER_FLAG} --upgrade tensorflow
! pip3 install {USER_FLAG} --upgrade matplotlib
! pip3 install {USER_FLAG} --upgrade numpy
! pip3 install {USER_FLAG} --upgrade pandas

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API, Cloud Build API, Cloud Storage API, and Container Registry API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudbuild.googleapis.com,storage.googleapis.com,containerregistry.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
  shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
  PROJECT_ID = shell_output[0]
  print("Project ID: ", PROJECT_ID)

Project ID:  


Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
  PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job, Vertex AI saves all resources to the given GCS bucket. We will also use the same bucket to download and host the input data. 

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants

In [ ]:
PIPELINE_NAME = '{0}-{1}'.format('tfp-anomaly-detection', TIMESTAMP)
PIPELINE_ROOT = '{0}/{1}'.format(BUCKET_NAME, PIPELINE_NAME)

In [ ]:
from typing import Callable, Optional, Mapping, Any

import kfp
from kfp.v2 import compiler
from kfp.v2 import dsl
from kfp.v2.google.client import AIPlatformClient
from kfp.v2.dsl import Input, Output, Dataset

### Define the anomaly detection components

Here you will load components from the [anomaly_detection](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection) folder in the [Google Cloud Pipeline Components SDK](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud).

You can also save and modify the original Python component file. For example, for [tfp_anomaly_detection.py](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/tfp_anomaly_detection.py):

* Call `generate_component_file()` which creates a yaml file.
* Replace the next cell with `anomaly_detection_op = kfp.components.load_component_from_file('component.yaml')`

The components do the following:
* `preprocess`: Regularizes and resamples a time series.
* `tfp_anomaly_detection`: Infers the structure of the time series, fits the model, and identifies anomalies based on the predictive distribution of acceptable values at each timestep.
* `postprocess`: Fills missing values from regularizing and resampling.

In [ ]:
preprocess_op = kfp.components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/preprocess.yaml')
anomaly_detection_op = kfp.components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/component.yaml')
postprocess_op = kfp.components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/postprocess.yaml')

### Define the pipeline

Here you will define the relationship between the components and how data is passed. In this pipeline a Google Cloud Storage csv is imported, the data is preprocessed, anomalies are flagged, and the results are postprocessed so that the output csv is scoreable by the Numenta Anomaly Benchmark.

In [ ]:
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT, name=PIPELINE_NAME)
def pipeline(input_url: str, memory_limit: str, seed: int) -> None:
  """
      Train model and return detected anomalies.
  """
  input_task = kfp.dsl.importer(
        artifact_uri=input_url,
        artifact_class=Dataset)
  preprocess_task = preprocess_op(input_dataset=input_task.output)
  anomaly_detection_task = anomaly_detection_op(input_dataset=preprocess_task.output, seed=seed).set_memory_limit(memory_limit)
  postprocess_op(input_dataset=input_task.output, predictions_dataset=anomaly_detection_task.output)

In [ ]:
def run_pipeline(pipeline: Callable,
                 parameter_values: Optional[Mapping[str, Any]] = {},
                 enable_caching: bool = False) -> None:
  """Runs a given pipeline function using Kubeflow Pipelines.

  Args:
   pipeline: The function to run.
   parameter_values: Parameters passed to the pipeline function when run.
   enable_caching: Whether to used cached results from previous runs.
  """
  compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='{}_pipeline.json'.format(PIPELINE_NAME))

  api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
  )

  _ = api_client.create_run_from_job_spec(
    job_spec_path='{}_pipeline.json'.format(PIPELINE_NAME),
    pipeline_root=PIPELINE_ROOT,
    parameter_values=parameter_values,
    enable_caching=enable_caching)

### Download the data

Here you will download the Numenta Anomaly Benchmark and upload the dataset to your GCS bucket. We will then find the exact GCS file url associated with the chosen task to pass as the input url into the pipeline.

In [ ]:
import os

NAB_DATA_BLOB = '{0}/NAB'.format(BUCKET_NAME)
if not os.path.exists('content/NAB'):
  !git clone https://github.com/numenta/NAB
!gsutil cp -r NAB/data $NAB_DATA_BLOB

In [ ]:
# Find the full file path in gcs for the chosen task
import tensorflow as tf

chosen_task_folder = 'realKnownCause'
chosen_task = 'nyc_taxi'
nab_files = tf.io.gfile.glob('{0}/*/*.csv'.format(NAB_DATA_BLOB))
chosen_task_file = [file for file in nab_files if chosen_task in file][0]
print('The pipeline will be run on the task: {0}'.format(chosen_task))

### Run the pipeline

Finally, we run the pipeline. Please wait until the run has completed before proceeding to the next steps.

In [ ]:
parameter_values = {
  'input_url': chosen_task_file,
  'memory_limit': '50G',
  'seed': 0,
}
run_pipeline(pipeline, parameter_values=parameter_values)

### Download the results locally

Copy the GCS file path from the final `postprocess` step of the pipeline below. Here we will save this output locally for visualization and scoring.

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
gcs_file = '[your-pipeline-output]' # @param {type:'string'}
output_file = '/content/{0}-{1}.csv'.format(chosen_task, TIMESTAMP)
!gsutil cp $gcs_file $output_file

In [ ]:
# Collect targets specifically for the chosen task
targets = json.load(open('/content/NAB/labels/combined_labels.json'))
chosen_task_targets = [targets[key] for key in targets if chosen_task in key][0]

### Visualize the results

Here we will plot the forecast distribution outputted by the pipeline, the points flagged as anomalies (red), and the ground truth targets (green). The graph is plotted with daily granularity due to the resampling done during preprocessing.

Note how the algorithm correctly identifies December 25th as an anomaly.

In [ ]:
#@title Plotting setup
from matplotlib import pylab as plt
from matplotlib.lines import Line2D

def plot_predictions(predictions: pd.DataFrame, annotation_fn: Callable = lambda timestamp: timestamp) -> None:
  """
    Plots the time series, forecast, detected anomalies, and residuals.

    Args:
      predictions: The output of the anomaly detection algorithm.
  """
  # Drop NaN values during plotting
  predictions = predictions.dropna(how='any')
  predictions = predictions.reset_index()

  timestamp = pd.to_datetime(predictions['timestamp'], format='%Y-%m-%d')
  # Plot the value from predictions which may be
  # an aggregation of the original value
  value = np.array(predictions['value_predictions'])
  lower_limit = np.array(predictions['lower_limit'])
  upper_limit = np.array(predictions['upper_limit'])
  mean = np.array(predictions['mean'])
  anomalies = np.array(predictions['label']).nonzero()[0]
  targets = []
  if 'target' in predictions:
    targets = np.array(predictions['target']).nonzero()[0]

  fig = plt.figure(figsize=(10, 5), constrained_layout=True)
  spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=[2., 1.])
  series_ax = fig.add_subplot(spec[0, 0])
  residuals_ax = fig.add_subplot(spec[1, 0], sharex=series_ax)

  # Plot anomalies on series_ax
  series_ax.plot(
      timestamp,
      value,
      color='black',
      alpha=0.6)
  series_ax.fill_between(
      timestamp,
      lower_limit,
      upper_limit,
      color='tab:blue',
      alpha=0.3)

  for anomaly_idx in anomalies:
    x = timestamp[anomaly_idx]
    y = value[anomaly_idx]
    series_ax.scatter(x, y, s=100, alpha=0.4, c='red')
  
  for target_idx in targets:
    x = timestamp[target_idx]
    y = value[target_idx]
    series_ax.scatter(x, y, s=100, alpha=0.4, c='green')
    series_ax.annotate(annotation_fn(x), (x, y))

  # Plot residuals on residuals_ax
  time_delta = timestamp[1] - timestamp[0]
  residuals_ax.bar(
      timestamp,
      height=upper_limit - lower_limit,
      bottom=lower_limit - mean,
      width=time_delta,
      align='center',
      color='tab:blue',
      alpha=0.3)
  residuals_ax.bar(
      timestamp,
      width=time_delta,
      height=value - mean,
      align='center',
      color='black',
      alpha=0.6)

  # Set up grid styling
  series_ax.set_ylabel('Original series')
  residuals_ax.set_ylabel('Residuals')
  series_ax.grid(True, color='whitesmoke')
  residuals_ax.grid(True, color='whitesmoke')
  series_ax.set_axisbelow(True)
  residuals_ax.set_axisbelow(True)

  # Add title and legend
  series_ax.set_title('TFP STS model forecast, anomalies, and residuals for {0}'.format(chosen_task))
  create_legend_label = lambda label, color: Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10)
  legend_elements = [create_legend_label(label, color) for label, color in [('predicted anomaly', 'red'), ('target', 'green')]]
  series_ax.legend(handles=legend_elements, loc='lower right')

In [ ]:
# Round target timestamps to day for plotting
round_to_day = lambda timestamp: timestamp.split()[0]
rounded_targets = [round_to_day(timestamp) for timestamp in chosen_task_targets]
rounded_targets = set(rounded_targets)
predictions = pd.read_csv(output_file)
predictions['target'] = predictions.apply(lambda df: round_to_day(df['timestamp']) in rounded_targets, axis=1)

In [ ]:
# Change the start and end to view different slices of the prediction
start, end = 8000, 9000
round_annotation = lambda timestamp: timestamp.date()
plot_predictions(predictions.iloc[start:end], round_annotation)

### Run scoring

Here we quantitatively score the algorithm's performance on the Numenta Anomaly Benchmark. The benchmark uses a custom scoring mechanism described in [their paper](https://arxiv.org/abs/1510.03336). Unlike precision and recall which do not reward for early detection, this scoring mechanism rewards based on windows around anomalous points rather than the exact points themselves.

We will run the scoring script with the `--optimize` flag, which uses the `anomaly_scores` column to score and optimizes the decision threshold. If this flag is omitted, then the script will only use the `label` column originally outputted by the component.

In [ ]:
# Set up NAB folder for running scoring
%cd /content/NAB
!pip install . --user
!python scripts/create_new_detector.py --detector $PIPELINE_NAME

In [ ]:
# Move gcs output into the NAB results folder structure
results_file = 'results/{0}/{1}/{0}_{2}.csv'.format(PIPELINE_NAME, chosen_task_folder, chosen_task)
!cp $output_file $results_file

In [ ]:
# Run the scoring script
!python run.py -d $PIPELINE_NAME --optimize --score --normalize

In [ ]:
#@title Score collection and normalization setup
import glob

def collect_scores(profile_name: str, chosen_task: str) -> pd.DataFrame:
  """Crawls through results files for all detectors in NAB to get results for the chosen task.
  
  Args:
    profile_name: One of 'standard', 'low_FP_rate', 'low_FN_rate'.
    chosen_task: The chosen benchmark task.
  
  Returns:
    all_scores_df: A pandas DataFrame of results for the task sorted by highest to lowest score.
  """
  all_scores = []
  for scores_file in glob.glob('/content/NAB/results/**/*_{0}_scores.csv'.format(profile_name)):
    scores_df = pd.read_csv(scores_file)
    chosen_task_row = scores_df[scores_df['File'].str.contains(chosen_task).fillna(False)]
    all_scores.append(chosen_task_row)
  all_scores_df = pd.concat(all_scores)
  all_scores_df = all_scores_df.sort_values(by=['Score'], ascending=False)
  all_scores_df = all_scores_df.reset_index().drop('index', axis=1)
  return all_scores_df

def normalize_scores(results: pd.DataFrame, profile_name: str, profiles: dict,
                     tpCount: int) -> pd.DataFrame:
  """Normalizes scores with the max from a perfect detector and the min from a null detector.

  Args:
    results: Pandas DataFrame with score results.
    profile_name: One of 'standard', 'low_FP_rate', 'low_FN_rate'.
    profiles: Dictionary containing cost matrix for each profile.
    tpCount: The number of true positives in the ground truth targets.

  Returns:
    The results DataFrame with an added column of normalized scores.
  """
  perfect = tpCount * profiles[profile_name]["CostMatrix"]["tpWeight"]
  # Note that the null detector's name is NaN in the `Detector` column
  base = results[pd.isna(results['Detector'])]['Score'].iloc[0]
  scores = results['Score']
  results['Normalized_Score'] = 100 * (scores - base) / (perfect - base)
  
  # Reindex column order for more organized table
  columns = results.columns.to_list()
  columns.remove('Score')
  columns.remove('Normalized_Score')
  columns += ['Score', 'Normalized_Score']
  results = results.reindex(columns=columns)
  print('Normalization used min raw score: {0} and max raw score: {1}'.format(base, perfect))
  return results

NAB also provides scores for three profile settings: `standard`, `reward_low_FN_rate`, and `reward_low_FP_rate`. If you run the cell below you can see the cost matrix for each profile, where `reward_low_FN_rate` penalizes false negatives more and `reward_low_FP_rate` penalizes false positives more. For example, if for the NYC Taxi & Limousine Commission it is worse to not have enough taxis during a big event than it is to have too many, then they may want to score based on a `reward_low_FN_rate` profile.

For the purposes of this demo we will only display results for the `standard` profile.



In [ ]:
tpCount = len(chosen_task_targets)
profile_name = 'standard'
profiles = json.load(open('/content/NAB/config/profiles.json'))
profiles

In [ ]:
results = collect_scores(profile_name, chosen_task)
results = normalize_scores(results, profile_name, profiles, tpCount)

In [ ]:
results